# Query the EFD

Craig Lage - 06-Aug-24

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.daf.butler import Butler

In [ ]:
client = makeEfdClient()
butler = Butler('LATISS', collections=['LATISS/raw/all', 
                                            'LATISS/calib/unbounded'])

# Sample query

In [ ]:
start = Time("2025-07-07 14:40:00Z", scale='utc')
end = Time("2025-07-07 15:15:00Z", scale='utc')
mount_position = getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", begin=start, end=end)

In [ ]:
mount_position['cRIO_timestamp'].head(5)

In [ ]:
from lsst.rubintv.production.mountTorques import calculateMountErrors
from logging import Logger

In [ ]:
logger = Logger("Test")

In [ ]:
expId = 2025070700003
instrument='LATISS'
where = "exposure.id=expId AND instrument='LATISS'"
expRecords = butler.registry.queryDimensionRecords(
            "exposure", where=where, bind={"expId": expId}, datasets="raw"
        )
filteredExpRecords = set(expRecords)
expRecord = filteredExpRecords.pop()


In [ ]:
figure = plt.figure(figsize=(10,12))
saveFilename = f"/home/cslage/DATA/{expId}_MountPlot.png"
calculateMountErrors(expRecord, client=client, butler=butler, logger=logger, figure=figure, saveFilename=saveFilename)

In [ ]:
figure.show()

In [ ]:
start = Time("2025-10-22 00:00:00Z", scale='utc')
end = Time("2025-10-22 21:00:00Z", scale='utc')
oods = getEfdData(client, "lsst.sal.MTOODS.logevent_imageInOODS", begin=start, end=end)
dayObs = 20251022
file = open(f"/home/cslage/DATA/Delays_{dayObs}.txt", 'w')
delays_1022 = []
for seqNum in range(1,66):
    obsid = f"MC_O_{dayObs}_{seqNum:06d}"
    this_image = oods[oods['obsid'] == obsid]
    times = this_image['private_kafkaStamp'].values
    #print(seqNum, len(times))
    delay = (np.max(times) - np.min(times))
    delays_1022.append(delay)
    file.write(f"For {obsid}, delay from first to last ingestion was {delay:.1f} seconds\n")
file.close()


In [ ]:

plt.subplot(1,2,1)
plt.title("20251022")
plt.hist(delays_1022, bins = 50, range=(0,100))
plt.subplot(1,2,2)
plt.title("20250921")
plt.hist(delays_0921, bins = 50, range=(0,100))

In [ ]:
plt.hist?

In [ ]:
test = oods[oods['obsid'] == obsid]
len(test)

In [ ]:
test['private_kafkaStamp']